# Preamble

In [1]:
import os, sys
import pandas as pd
import numpy as np

In [2]:
sys.path.append('/Users/lindenmp/Dropbox/Work/ResProjects/NormativeNeuroDev_CrossSec/code/func/')
from proj_environment import set_proj_env
from func import get_synth_cov

In [3]:
train_test_str = 'squeakycleanExclude'
exclude_str = 't1Exclude' # 't1Exclude' 'fsFinalExclude'
parc_str = 'schaefer' # 'schaefer' 'lausanne'
parc_scale = 400 # 200 400 | 60 125
extra_str = ''
# extra_str = '_nuis-netdens'
# extra_str = '_nuis-str'
parcel_names, parcel_loc, drop_parcels, num_parcels, yeo_idx, yeo_labels = set_proj_env(train_test_str = train_test_str, exclude_str = exclude_str,
                                                                            parc_str = parc_str, parc_scale = parc_scale, extra_str = extra_str)

In [4]:
print(os.environ['MODELDIR_BASE'])
print(os.environ['MODELDIR'])

/Users/lindenmp/Dropbox/Work/ResProjects/NormativeNeuroDev_CrossSec/analysis/normative/t1Exclude/squeakycleanExclude/schaefer_400_streamlineCount
/Users/lindenmp/Dropbox/Work/ResProjects/NormativeNeuroDev_CrossSec/analysis/normative/t1Exclude/squeakycleanExclude/schaefer_400_streamlineCount


## Load data

In [5]:
# Load data
df = pd.read_csv(os.path.join(os.environ['MODELDIR_BASE'], 'df_pheno.csv'))
df.set_index(['bblid', 'scanid'], inplace = True)

df_node = pd.read_csv(os.path.join(os.environ['MODELDIR'], 'df_node_clean.csv'))
df_node.set_index(['bblid', 'scanid'], inplace = True)

# adjust sex to 0 and 1
df['sex_adj'] = df.sex - 1
print(df.shape)
print(df_node.shape)

(1100, 21)
(1100, 1601)


In [6]:
df.head()

,,squeakycleanExclude,ageAtScan1,ageAtScan1_Years,sex,race2,handednessv2,dti64MeanAbsRMS,dti64MeanRelRMS,dti64MaxAbsRMS,dti64MaxRelRMS,...,averageManualRating,Overall_Psychopathology,Psychosis_Positive,Psychosis_NegativeDisorg,AnxiousMisery,Externalizing,Fear,streamline_count,network_density,sex_adj
bblid,scanid,,,,,,,,,,,,,,,,,,,,,
81287,2738,0,240,20.0,2,1,1,0.351665,0.154478,0.557358,0.272518,...,2.0,0.314394,-1.109349,-0.572468,1.222410,1.134094,-0.235063,294415.0,0.064173,1
80680,2739,1,253,21.1,1,1,1,0.531246,0.208078,0.754717,0.354560,...,2.0,0.748088,-0.905436,1.414109,0.032372,2.343126,0.744692,280623.0,0.057820,0
81754,2740,1,232,19.3,2,1,1,0.310943,0.218462,0.460633,0.387235,...,2.0,1.381240,0.986549,-0.969899,0.301802,-0.313214,-0.344655,269473.0,0.067594,1
81903,2749,0,231,19.2,2,1,1,0.432610,0.283153,0.819576,0.509537,...,2.0,0.531327,-0.885234,0.085093,1.994165,-0.831330,-1.902806,224511.0,0.060877,1
81043,2750,0,249,20.8,2,2,1,0.162409,0.096761,0.429102,0.259678,...,2.0,-0.330777,-0.788469,-0.798989,-2.036093,-0.072305,-0.746145,270865.0,0.065451,1


In [7]:
df_node.head()

,,squeakycleanExclude,ct_0,ct_1,ct_2,ct_3,ct_4,ct_5,ct_6,ct_7,ct_8,...,mc_390,mc_391,mc_392,mc_393,mc_394,mc_395,mc_396,mc_397,mc_398,mc_399
bblid,scanid,,,,,,,,,,,,,,,,,,,,,
81287,2738,0,-0.387087,0.004407,-0.267396,-0.028519,-0.339460,-0.222917,-0.555840,-0.554495,-0.403831,...,0.029287,0.010829,-0.003852,0.019600,0.006431,0.016437,0.000043,0.013576,0.002063,0.015741
80680,2739,1,0.234626,-0.224065,-0.170527,-0.193183,0.282391,-0.087519,0.163466,0.131132,0.250391,...,0.033283,-0.028287,0.004943,0.002286,-0.000051,0.009080,-0.002439,-0.016799,0.000636,0.011507
81754,2740,1,0.031087,0.452194,0.682236,0.294448,0.417452,0.303948,0.855036,-0.007639,0.199695,...,0.031682,0.051293,0.006959,0.041199,0.009133,0.021211,0.002054,0.015383,0.007908,0.010865
81903,2749,0,0.035996,-0.045988,0.318626,-0.063466,-0.001922,0.035968,-0.422805,0.314472,0.362054,...,0.010152,0.022177,0.005587,0.032746,0.005475,0.013376,0.001784,0.018394,0.008128,0.005626
81043,2750,0,-0.444289,-0.525050,0.189272,-0.571304,-0.086752,-0.360070,-0.725818,0.813983,0.604498,...,-0.051749,0.033476,-0.000507,-0.022667,0.002194,0.002912,-0.002322,0.015815,-0.000447,0.006430


# Prepare files for normative modelling

In [8]:
# Note, 'ageAtScan1_Years' is assumed to be covs[0] and 'sex_adj' is assumed to be covs[1]
# if more than 2 covs are to be used, append to the end and age/sex will be duplicated accordingly in the forward model
covs = ['ageAtScan1_Years', 'sex_adj']

print(covs)
num_covs = len(covs)
print(num_covs)

['ageAtScan1_Years', 'sex_adj']
2


In [9]:
extra_str_2 = ''

## Primary model (train/test split)

In [10]:
# Create subdirectory for specific normative model -- labeled according to parcellation/resolution choices and covariates
normativedir = os.path.join(os.environ['MODELDIR'], '+'.join(covs) + extra_str_2 + '/')
print(normativedir)
if not os.path.exists(normativedir): os.mkdir(normativedir);

/Users/lindenmp/Dropbox/Work/ResProjects/NormativeNeuroDev_CrossSec/analysis/normative/t1Exclude/squeakycleanExclude/schaefer_400_streamlineCount/ageAtScan1_Years+sex_adj/


In [11]:
# Write out training -- retaining only residuals from nuissance regression
df[df[train_test_str] == 0].to_csv(os.path.join(normativedir, 'train.csv'))
df[df[train_test_str] == 0].to_csv(os.path.join(normativedir, 'cov_train.txt'), columns = covs, sep = ' ', index = False, header = False)

resp_train = df_node[df_node[train_test_str] == 0].drop(train_test_str, axis = 1)
mask = np.all(np.isnan(resp_train), axis = 1)
if np.any(mask): print("Warning: NaNs in response train")
resp_train.to_csv(os.path.join(normativedir, 'resp_train.csv'))
resp_train.to_csv(os.path.join(normativedir, 'resp_train.txt'), sep = ' ', index = False, header = False)

# Write out test -- retaining only residuals from nuissance regression
df[df[train_test_str] == 1].to_csv(os.path.join(normativedir, 'test.csv'))
df[df[train_test_str] == 1].to_csv(os.path.join(normativedir, 'cov_test.txt'), columns = covs, sep = ' ', index = False, header = False)

resp_test = df_node[df_node[train_test_str] == 1].drop(train_test_str, axis = 1)
mask = np.all(np.isnan(resp_test), axis = 1)
if np.any(mask): print("Warning: NaNs in response train")
resp_test.to_csv(os.path.join(normativedir, 'resp_test.csv'))
resp_test.to_csv(os.path.join(normativedir, 'resp_test.txt'), sep = ' ', index = False, header = False)

print(str(resp_train.shape[1]) + ' features written out for normative modeling')

1600 features written out for normative modeling


### Forward variants

In [12]:
fwddir = os.path.join(normativedir, 'forward/')
if not os.path.exists(fwddir): os.mkdir(fwddir)

# Synthetic cov data
x = get_synth_cov(df, cov = 'ageAtScan1_Years', stp = 1)

if 'sex_adj' in covs:
    # Produce gender dummy variable for one repeat --> i.e., to account for two runs of ages, one per gender
    gender_synth = np.concatenate((np.ones(x.shape),np.zeros(x.shape)), axis = 0)

# concat
synth_cov = np.concatenate((np.matlib.repmat(x, 2, 1), np.matlib.repmat(gender_synth, 1, 1)), axis = 1)
print(synth_cov.shape)

# write out
np.savetxt(os.path.join(fwddir, 'synth_cov_test.txt'), synth_cov, delimiter = ' ', fmt = ['%.1f', '%.d'])

(30, 2)


### Permutation test | train and test | no blocks

In [13]:
# number of permutations
num_perms = 1000

# Set seed for reproducibility
np.random.seed(0)

for i in range(num_perms):
    permdir = os.path.join(normativedir, 'perm_all/perm_' + str(i))
    if not os.path.exists(permdir): os.makedirs(permdir)

    df_shuffed = df.copy()
    df_shuffed.loc[:,covs] = df_shuffed[covs].sample(frac = 1).values
    df_shuffed.loc[:,covs[1]] = df_shuffed.loc[:,covs[1]].astype(int)

    df_shuffed[df_shuffed[train_test_str] == 0].to_csv(os.path.join(permdir, 'cov_train.txt'), columns = covs, sep = ' ', index = False, header = False)
    df_shuffed[df_shuffed[train_test_str] == 1].to_csv(os.path.join(permdir, 'cov_test.txt'), columns = covs, sep = ' ', index = False, header = False)